----------------------------------------------------
Modelagem de tópicos de documentos provenientes da base de dados do CPDOC.
----------------------------------------------------
Base de dados: Antonio Azeredo da Silveira.
-----------------------------
Número de documentos: 10.428
-----------------------------
Número de imagens: 66.938
-----------------------------

### Topic: Clustering documents

## Contents

<ul>
<li>[Stopwords, stemming, and tokenization](#Stopwords,-stemming,-and-tokenizing)
<li>[Tf-idf and document similarity](#Tf-idf-and-document-similarity)
<li>[K-means clustering](#K-means-clustering)
<li>[Multidimensional scaling](#Multidimensional-scaling)
<li>[Visualizing document clusters](#Visualizing-document-clusters)
<li>[Hierarchical document clustering](#Hierarchical-document-clustering)
<li>[Latent Dirichlet Allocation (LDA)](#Latent-Dirichlet-Allocation)
</ul>

But first, I import everything I am going to need up front

In [1]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import feature_extraction
import mpld3
import pyLDAvis
import pyLDAvis.gensim

%matplotlib inline

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead

Specifying the path to the files  

In [2]:
#datapath = 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected' # path marcelo
datapath = "../textfiles-corrected"
outputs = "../outputs/"
import os

onlyfiles = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath,f))]

In [3]:
onlyfiles[0:4]

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):


['txt_0002560_270.txt',
 'txt_0000001_152.txt',
 'txt_0004608_357.txt',
 'txt_0005120_486.txt']

#### make list of txt contents of files

In [5]:
count = 0
percentil = int(len(onlyfiles)/100)
textos = []

for txt in onlyfiles[0:]:
    with open(os.path.join(datapath,txt), 'r', encoding='utf-8') as f:
        count += 1
        if count % percentil == 0: print(int(count/percentil),'% done')
        texto = f.read()
        texto = texto.split()
        textos.append(texto)
#print(textos)

1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done
81 % done
82 % done
83 % done
84 % done
85 % done
86 % done
87 % done
88 % done
89 % done
90 % done
91 % done
92 % done
93 % done
94 % done
95 % done
96 % done
97 % done
98 % done
99 % done
100 % done


### Stopwords, stemming, and tokenizing

This section is focused on defining some functions to manipulate the synopses. First, I load [NLTK's](http://www.nltk.org/) list of English stop words. [Stop words](http://en.wikipedia.org/wiki/Stop_words) are words like "a", "the", or "in" which don't convey significant meaning. I'm sure there are much better explanations of this out there.

In [6]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('portuguese')

### Latent Dirichlet Allocation

In [7]:
#Latent Dirichlet Allocation implementation with Gensim
from gensim import corpora, models, similarities 

%time texts = [[word for word in text if word not in stopwords] for text in textos]

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/theano/configparser.py:81: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  'PID': str(os.getpid()),
Using gpu device 0: Quadro K4200 (CNMeM is disabled, cuDNN 5103)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


CPU times: user 6.99 s, sys: 28 ms, total: 7.02 s
Wall time: 7.01 s


In [8]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

174


In [9]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
corpus = [dictionary.doc2bow(text) for text in texts]

In [10]:
len(corpus)

6318

In [ ]:
%time lda = models.LdaModel(corpus, num_topics=10, id2word=dictionary, update_every=5, chunksize=10000, passes=10)

In [ ]:
print(lda[corpus[0]])

In [ ]:
topics = lda.print_topics(5, num_words=20)

In [ ]:
lda.show_topics()

In [ ]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)

In [ ]:
topics_matrix[0]

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)